In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [8]:
links = None

with open('links.json', 'r') as f:
    links = json.loads(f.read())

links

{'Hindi-Persian': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian'}

In [9]:
base_link = "https://en.m.wiktionary.org"


In [10]:
def get_words_ipas(dest_lang, src_lang, link, page_num):
    borrowed_words = []

    try:
        # get list of words
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        res = soup.find_all("div", class_="mw-category-group")

        for i, r in enumerate(res):
            if i > 0:
                list_words = r.find_all('a')
                src_word = None

                for borrowed_word in list_words:

                    # for each loaned word get the wiki link and extract ipas for it.
                    ipa_page = requests.get(base_link + borrowed_word['href'])
                    ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')
                    
                    
                    # get the src word and wiki link to extract ipas for the src word
                    src_word_html_final = None
                    for src_word_html in ipa_soup.find_all("span", class_="etyl"):
                        if src_word_html != None:
                            for tag in src_word_html.find(lambda tag: tag.name == 'a') :
                                if src_lang.lower() not in tag.text.lower():
                                    src_word_html_final = src_word_html.find_next("i")
                                    src_word = src_word_html_final.text if src_word_html_final != None else ''

                                    borrowed_words.append( [borrowed_word.text, src_word,tag.text.lower()]) 
                                # src_word_link = src_word_html_final.find('a')
                                
  
                    # borrowed_words.append([borrowed_word.text, src_word])

        print(dest_lang, src_lang, page_num, "done", len(borrowed_words))
        # return borrowed_words      
        # find whet her there is a next page containing more loan words if there is scrape that as well. 
        next_link = soup.find(
            lambda tag: tag.name == 'a' and 'next page' in tag.text.lower()
        )

        # return borrowed_words
        if next_link != None:
            more_words = get_words_ipas(dest_lang, src_lang, base_link + next_link['href'], page_num+1)
            borrowed_words = borrowed_words + more_words

    except Exception as e:
        # errors can be due to timeouts/connections refused due to rate limiting. can be fixed via proxies/vpns/timeouts
        print(pair, page_num, "got error", e, len(borrowed_words))
        return borrowed_words
        
    return borrowed_words

In [11]:
for pair in links:
    [dest, src] = pair.split('-')
    words = get_words_ipas(dest, src, links[pair], 1)
    df = pd.DataFrame(words, columns=['loan_word', 'original_word', 'other_etymology'])
    df.to_csv("results/{}_false_friends.csv".format(pair), index=False)
    print(pair, "done")



Hindi Persian 1 done 220
Hindi Persian 2 done 160
Hindi Persian 3 done 197
Hindi Persian 4 done 195
Hindi Persian 5 done 194
Hindi Persian 6 done 199
Hindi Persian 7 done 253
Hindi-Persian done
